# Chemical equilibrium with fixed fugacity

This tutorial demonstrates the calculation of calcite (CaCO<sub>3</sub>) solubility in a saline aqueous solution for **given fugacity of CO<sub>2</sub>**.

We consider a chemical system containing an aqueous phase, representing the brine, and a solid mineral phase to model our calcite mineral.

In [1]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C Ca"))
solution.setActivityModel(ActivityModelHKF())

calcite = MineralPhase("Calcite")

system = ChemicalSystem(db, solution, calcite)

In chemical equilibrium calculation, the following properties must be constrained:

* temperature,
* pressure, and
* fugacity of CO<sub>2</sub>.

They are specified with the class [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) and then initialized with given value in [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html):

In [2]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2(g)")

solver = EquilibriumSolver(specs)

fCO2g = 0.1  # 0.1 bar

conditions = EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.fugacity("CO2(g)", fCO2g, "bar")

> **Note**: Because the fugacity of CO<sub>2</sub> is constrained, the chemical equilibrium problem will presume that the **system is open to CO<sub>2</sub>** (enough CO<sub>2</sub> is allowed to leave/enter the system to satisfy the fugacity constraint).

Next, we create an initial chemical state representing a 1 molal NaCl aqueous solution mixed with 10 g of calcite mineral and perform equilibration:

In [3]:
state = ChemicalState(system)
state.temperature(50.0, "celsius")
state.pressure(10.0, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("Calcite", 10, "g")

result = solver.solve(state, conditions)
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

print("EQUILIBRIUM STATE")
print(state)

Successful computation!
EQUILIBRIUM STATE
+---------------------+-------------+------+
| Property            |       Value | Unit |
+---------------------+-------------+------+
| Temperature         |    323.1500 |    K |
| Pressure            |     10.0000 |  bar |
| Charge:             | -1.1210e-16 |  mol |
| Element Amount:     |             |      |
| :: H                |  1.1101e+02 |  mol |
| :: C                |  1.0692e-01 |  mol |
| :: O                |  5.5820e+01 |  mol |
| :: Na               |  1.0000e+00 |  mol |
| :: Cl               |  1.0000e+00 |  mol |
| :: Ca               |  9.9909e-02 |  mol |
| Species Amount:     |             |      |
| :: CO3-2            |  1.5610e-05 |  mol |
| :: H+               |  2.5243e-07 |  mol |
| :: H2O              |  5.5501e+01 |  mol |
| :: CO2              |  1.5215e-03 |  mol |
| :: (CO2)2           |  1.0000e-16 |  mol |
| :: HCO3-            |  9.2960e-03 |  mol |
| :: CH4              |  1.0000e-16 |  mol |
| :: Ca+2    

Note in the table, that calcite was not fully dissolved with the specified conditions, and the solution is thus saturated with it.

A more in-depth look at the chemical state of the system might be obtained from generating chemical properties  using the class [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html):

In [4]:
props = ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |     323.1500 |         K |
| Pressure                               |      10.0000 |       bar |
| Volume                                 |   1.0330e-03 |        m3 |
| Gibbs Energy                           |   -5891.6218 |         J |
| Enthalpy                               |    1596.7091 |         J |
| Entropy                                |      23.1729 |       J/K |
| Internal Energy                        |     563.7590 |         J |
| Helmholtz Energy                       |   -6924.5719 |         J |
| Charge                                 |  -1.1210e-16 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.1101e+02 |       mol |
| :: C              

For instance, this table shows the activities of species, their chemical potentials, and standard thermodynamic properties.

The class [AqueousProps](https://reaktoro.org/api/classReaktoro_1_1AqueousProps.html) can be used to study properties pertinent to aqueous solutions:

In [5]:
aprops = AqueousProps(state)
print(aprops)

+-----------------------------------+------------+-------+
| Property                          |      Value |  Unit |
+-----------------------------------+------------+-------+
| Temperature                       |   323.1500 |     K |
| Pressure                          |    10.0000 |   bar |
| Ionic Strength (Effective)        |     1.0152 | molal |
| Ionic Strength (Stoichiometric)   |     1.0167 | molal |
| pH                                |     6.7952 |       |
| pE                                |    -2.5006 |       |
| Eh                                |    -0.1603 |     V |
| Element Molality:                 |            |       |
| :: C                              | 1.2680e-02 | molal |
| :: Na                             | 1.0001e+00 | molal |
| :: Cl                             | 1.0001e+00 | molal |
| :: Ca                             | 5.6644e-03 | molal |
| Species Molality:                 |            |       |
| :: CO3-2                          | 1.5612e-05 | molal

Let's now find out if our computed chemical equilibrium state satisfies the imposed fugacity constraint. In other words, the following must be satisfied:

$$\mu_{\mathrm{CO_{2}(aq)}}=\mu_{\mathrm{CO_{2}(g)}},$$

where

$$\mu_{\mathrm{CO_{2}(g)}}\equiv\mu_{\mathrm{CO_{2}(g)}}^{\circ}+RT\log f_{\mathrm{CO_{2}(g)}}.$$

Here,

* $\mu_{\mathrm{CO_{2}(aq)}}$, the *chemical potential of aqueous species CO<sub>2</sub>*, can be obtained from the [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) object,
* $\mu_{\mathrm{CO_{2}(g)}}^{\circ}$, the *standard chemical potential of the gaseous species CO<sub>2</sub>(g)*,  that can be obtained from the *standard molar Gibbs energy* $G_{\mathrm{CO_{2}(g)}}^{\circ}$ stored in the thermodynamic properties of species in a used database;
* $f_{\mathrm{CO_{2}(g)}}$ is the *imposed fugacity* value for CO<sub>2</sub>(g).

The code below verifies this identity:

In [6]:
from math import log

R = universalGasConstant
T = props.temperature()
P = props.pressure()

uCO2aq = props.speciesChemicalPotential("CO2")
u0CO2g = db.species().get("CO2(g)").props(T, P).G0
uCO2g  = u0CO2g + R*T*log(fCO2g)

print(f"μCO2(aq) = {uCO2aq} J/mol")
print(f"μCO2(g)  = {uCO2g} J/mol")

μCO2(aq) = -118583 J/mol
μCO2(g)  = -118583 J/mol


The identity of μ<sub>CO2(aq)</sub> and μ<sub>CO2(aq)</sub> means that the fugacity constraint was satisfied.

Related application tutorials:

* [Phosphate accumulation in carbonate-rich brines](tutorials/geobiology/geobiology-phreeqc-fixed-fugacity.ipynb)
* [Carbonate-rich lakes modelling on the early Earth](tutorials/geobiology/geobiology-streammodel-fixed-fugacity.ipynb)